# the `Result` module

As of this writing, the `Result` module [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-core-resultmodule.html)] has three functions:

1. `Result.bind`
2. `Result.map`
3. `Result.mapError`

These functions serve to reduce the monotony of three kinds of `match` expressions around `Result<T',TError'>`, respectively:

1. bind to a function that returns `Result<T',TError'>` to yield an `Ok` or pass through `Error err`
2. yield an `Ok` or pass through `Error err` from an instance of `Result<T',TError'>`
3. yield an `Error` or pass through `Ok value` from an instance of `Result<T',TError'>`

The follwing .NET `JsonDocument` represents input that can be deconstructed into results of `Result<JsonElement, Exception>`:

In [ ]:
#!fsharp

open System.Text.Json

let input = JsonDocument.Parse("""
        {
            "root": {
                "first": {
                    "firstChild": 2.5
                },
                "second": {
                    "secondChild": true
                },
                "fourth": {
                    "thirdChild": false
                }
            }
        }
    """)


We can define a general-purpose utility function that can _try_ to _get_ JSON document properties as `JsonElement`:

In [ ]:
#!fsharp

let tryGetProperty (elementName: string) (element: JsonElement) =
    match element.TryGetProperty elementName with
    | false, _ -> Error(JsonException $"the expected `{elementName}` element is not here.")
    | true, el -> Ok el


With naïve determination, we can take the path of the honorable novice and use `match`:

In [ ]:
#!fsharp

match input.RootElement |> tryGetProperty "root" with
| Error err -> Error err
| Ok rootElement ->
    match rootElement |> tryGetProperty "second" with
    | Error err -> Error err
    | Ok secondElement ->
        match secondElement |> tryGetProperty "secondChild" with
        | Error err -> Error err
        | Ok secondChildElement -> Ok (secondChildElement.GetBoolean())


ResultValue,ErrorValue
True,<null>


But we notice that every `match` is yielding `Error err` and `Ok (…)`. This is the `match`-yield pattern that `Result.bind` is designed for:

In [ ]:
#!fsharp

let secondChildResult =
    input.RootElement
    |> tryGetProperty "root"
    |> Result.bind (tryGetProperty "second")
    |> Result.bind (tryGetProperty "secondChild")

secondChildResult

ResultValue,ErrorValue
True,<null>


We can see why Scott Wlaschin uses `Result.bind` this way in the context of validation as a subtopic of [Railway Oriented Programming](https://swlaschin.gitbooks.io/fsharpforfunandprofit/content/posts/recipe-part2.html). An unlimited number of `try*` functions can be used to _try_ validation rules on `input`.

## when to use `Result.map`

Notice how `Result.bind` needs a `try*` function. How can we return a `Result<_,_>` without a `try*` function but still have the convenience of `Result.bind`?

The answer is to use `Result.map`. But before we do, we can, again, take the path of the honorable novice and turn to `match`:

In [ ]:
#!fsharp

match secondChildResult with
| Error err -> Error err
| Ok secondChildElement -> Ok (secondChildElement.GetBoolean())

ResultValue,ErrorValue
True,<null>


Now compare to `Result.map`:

In [ ]:
#!fsharp

secondChildResult
|> Result.map (fun secondChildElement -> secondChildElement.GetBoolean())

ResultValue,ErrorValue
True,<null>


We see that `Result.map` allows us to:

1. apply a function to the `Ok` `JsonElement`
2. get rid of the `Error err -> Error err` pattern

## when to use `Result.mapError`

When we need to apply a function to `TError'` of `Result<T',TError'>`, we use `Result.mapError`.

In [ ]:
#!fsharp

secondChildResult
|> Result.mapError (fun r -> r |> printfn "my-logger: %A"; r)
|> Result.map (fun secondChildElement -> secondChildElement.GetBoolean())

ResultValue,ErrorValue
True,<null>


The elegance of `Result.mapError` is in the form of its unobtrusiveness: it just lets the `Result<_,_>` pass through when there is no error.

But when there _is_ an error it takes action:

In [ ]:
#!fsharp

input.RootElement
|> tryGetProperty "frankie"
|> Result.mapError (fun ex -> ex |> printfn "my-logger: %A"; ex)
|> Result.map (fun el -> el.GetBoolean())

my-logger: System.Text.Json.JsonException: the expected `frankie` element is not here.


ResultValue,ErrorValue
False,"{ System.Text.Json.JsonException: the expected `frankie` element is not here.: LineNumber: <null>, BytePositionInLine: <null>, Path: <null>, Message: the expected `frankie` element is not here., TargetSite: <null>, Data: [ ], InnerException: <null>, HelpLink: <null>, Source: <null>, HResult: -2146233088, StackTrace: <null> }"


`Result.mapError` mapped the erroneous search for a JSON element named `"frankie"` to the logger and then passed it on to `Result.map` which left the Boolean value of `T'` defaulted to `false`.

## the `Result` module does not have a function for getting the `Ok` value

As of this writing, the `Result` module depends on `match` expressions to yield the `Ok` value:

In [ ]:
#!fsharp

let outputResult =
    input.RootElement
    |> tryGetProperty "root"
    |> Result.bind (tryGetProperty "second")
    |> Result.bind (tryGetProperty "secondChild")
    |> Result.map (fun secondChildElement -> secondChildElement.GetBoolean())

match outputResult with
| Ok output -> output
| _ -> false

True

Without the help of third-party libraries like `FsToolKit.ErrorHandling` [[GitHub](https://github.com/demystifyfp/FsToolkit.ErrorHandling#readme)], we must resort to the honorable `match` expression 🤠

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
